In [1]:
#;.pykx.disableJupyter()

In [2]:
# https://code.kx.com/pykx/3.0/examples/jupyter-integration.html#q-first-mode
import pykx as kx
kx.util.jupyter_qfirst_enable()

PyKX now running in 'jupyter_qfirst' mode. All cells by default will be run as q code. 
Include '%%py' at the beginning of each cell to run as python code. 


## KX Training Attributes: Exercises

* * Create a list of 200000 distinct numbers. Measure the time to search for items at the front, middle and back of the list repeating the seach 1000 times.

In [13]:
x:til 200000
front:x[0]     //getting first value 
middle:x[`int$count[x]%2]  //getting middle value
back:x[-1+count[x]]   //getting last value

\t:1000 x?front
\t:1000 x?middle
\t:1000 x?back

0
37
83


In [4]:
//write your code here
intlist:til 200000

front:intlist[0]
middle:intlist[`int$count[intlist]%2]
end:intlist[-1+count[intlist]]

\ts:1000 intlist?front
\ts:1000 intlist?middle
\ts:1000 intlist?end

0 704
38 704
78 704


* Repeat the above question however apply the sorted attribute to the list

In [14]:
sx:`s#x

\t:1000 sx?front
\t:1000 sx?middle
\t:1000 sx?back

0
0
0


In [5]:
//write your code here
new:`s#intlist
\ts:1000 new?front
\ts:1000 new?middle
\ts:1000 new?end

0 704
0 704
0 704


* Repeat again and apply the unique attribute to the list:

In [ ]:
ux:`u#x

\t:1000 ux?front
\t:1000 ux?middle
\t:1000 ux?back

In [6]:
//write your code here
newUnique:`u#intlist

\t:1000 newUnique?front
\t:1000 newUnique?middle
\t:1000 newUnique?end

0
0
0


* Show the memory overhead using [hcount](https://code.kx.com/q/ref/hcount/) on the below lists with a grouped attribute:
     1. List of longs
     2. List of ints

In [ ]:
`:listOfLong set til 100000       //saving it down as a file on disk
`:listOfGroupLong set `g#til 100000

hcount `:listOfLong
hcount `:listOfGroupLong
hcount[`:listOfGroupLong] - hcount `:listOfLong //finding the difference in bytes

In [ ]:
`:listOfInt set `int$til 100000        //saving it down as a file on disk
`:listOfGroupInt set `g#`int$til 100000

hcount `:listOfInt
hcount `:listOfGroupInt
hcount[`:listOfGroupInt]- hcount `:listOfInt //finding the difference in bytes

In [7]:
//write your code here
`:longList set til 100000
`:GroupedLongList set `g#til 100000

show a:hcount `:longList
show b:hcount `:GroupedLongList

abs a-b

//GROUPED HAS A LOT OF OVERHEAD

:longList
:GroupedLongList
4745784
800016
5545800


In [24]:
`:integerList set `int$til 100000
`:GroupedintegerList set `g#`int$til 100000

show c:hcount `:integerList
show d:hcount `:GroupedintegerList

abs c-d

//GROUPED HAS A LOT OF OVERHEAD

400016
4621512
:integerList
:GroupedintegerList
4221496


* What would be the conditions required for a table to be a good candidate to have the sorted attribute applied to a column of a table?

Static table or guaranteed that all inserts preserve the sort. If this condition is met there is no point not having it – no downside, even if the sort is preserved infrequently. For example, the time column of the trade table in the standard tick configuration, provided all the timestamps come from a single ticker plant (time monotonicity is preserved)

In [8]:
//no coding is required for this question

//The condition is that whatever the new entries are, they should preserve the sorted attribute

Let's define a small trades table for the next question.

In [9]:
n:100000
show 5#smalltrade:([]date:n#.z.d;time:09:30:00.000+n?08:00:00;sym:n?`AIG`AAPL`DELL`DOW`GOOG`IBM;price:n?100.)

date       time         sym  price   
-------------------------------------
2025.03.13 11:30:39.000 AIG  45.25059
2025.03.13 14:43:30.000 DELL 50.60316
2025.03.13 15:53:32.000 DOW  57.56219
2025.03.13 09:44:38.000 GOOG 91.1725 
2025.03.13 13:59:04.000 AAPL 42.95208


* Using tables that has the sorted attribute and no attribute applied to the time column, which table retrieves the below query the fastest repeating 100 times :

```select from smalltrade where time within 12:00 13:00```

In [ ]:
sortedtrade:`time xasc smalltrade

\t:100 select from smalltrade where time within 12:00 13:00
\t:100 select from sortedtrade where time within 12:00 13:00 //using table with sorted attribute

In [10]:
\t:100 select from smalltrade where time within 12:00 13:00

27


In [11]:
//write your code here
sortedTable:`time xasc smalltrade
\t:100 select from sortedTable where time within 12:00 13:00


4


* What would be the conditions required for a table to be a good candidate to have the unique attribute applied to a column of a table?

Either a static table where the uniqueness of the column is preserved, or a single keyed table where uniqueness is automatically enforced. No real down side to applying the unique attribute to any column of a table with length > 100 (approx). For example, a keyed table containing instrument information on a per stock basis.

In [12]:
//no coding is required for this question
/
Whatever the new entries are in the column, they must have not been in the column before. They must be unique. It can also be a keyed column where uniqueness is 
autoimatically enforced.
\

* What would be the conditions required for a table to be a good candidate to have the grouped attribute applied to a column of a table?

Any column with a large number of entries but a much smaller number of distinct items which is regularly searched. The downside is extra memory usage. For example, the sym column of the trade table in the RTD of a standard tick db.


In [ ]:
//no coding is required for this question
//If a column has a large number of entries, but has a smaller number of unique items that are searched frequently

* Demonstrate using a table the difference made to search times by having the grouped attribute on sym using the query below:

``` select from smalltrade where sym=`IBM ```

In [ ]:
groupedtrade: update `g#sym from smalltrade

\t:100 select from smalltrade where sym=`IBM
\t:100 select from groupedtrade where sym=`IBM

In [14]:
//write your code here
groupTrade: update `g#sym from smalltrade
\t:1000 select from smalltrade where sym = `IBM
\t:1000 select from groupTrade where sym = `IBM 

220
108


Given the table below, implement three ways to speed up queries on the data.

* **Query**- select avg price from t where sym=`JPM,time.time>09:00,ID within 500 600

In [15]:
n:1000000
show t:([]time:"P"$(string[.z.d],"D"),/:string[n?00:00:00.000];sym:n?`AAPL`JPM`GOOG`AMZN;ID:"i"$neg[n]?n;price:n?100f)

time                          sym  ID     price   
--------------------------------------------------
2025.03.13D22:35:09.792000000 JPM  491599 33.7496 
2025.03.13D11:08:16.049000000 JPM  508615 28.70688
2025.03.13D15:50:11.639000000 JPM  9503   64.45635
2025.03.13D18:18:19.684000000 GOOG 769170 96.59235
2025.03.13D19:54:59.594000000 AMZN 463077 15.93356
2025.03.13D17:28:21.697000000 AMZN 31     89.97697
2025.03.13D10:46:42.881000000 JPM  403657 50.27151
2025.03.13D03:50:14.483000000 JPM  659198 85.36489
2025.03.13D01:12:12.009000000 GOOG 868970 41.5535 
2025.03.13D01:09:55.973000000 AMZN 36549  11.1922 
2025.03.13D18:08:05.162000000 AAPL 595205 35.97477
2025.03.13D14:27:42.086000000 AMZN 284591 33.59551
2025.03.13D16:32:54.785000000 JPM  737551 4.047001
2025.03.13D20:09:41.823000000 AMZN 539407 18.84824
2025.03.13D01:10:00.559000000 JPM  200329 81.28662
2025.03.13D11:20:37.273000000 GOOG 800948 95.72724
2025.03.13D22:02:17.172000000 AMZN 775578 40.83471
2025.03.13D15:02:20.336000000 A

In [21]:
newTab:`time xasc t //speeding up the time lookup using the sorted attribute on time column
newTab:update `u#ID, `g#sym from newTab //applying grouped and unique to improve performance on sym and ID lookup

In [22]:
\t:1000 select avg price from t where sym=`JPM,time.time>09:00,ID within 500 600
\t:1000 select avg price from newTab where sym=`JPM,time.time>09:00,ID within 500 600

7610
6814


In [19]:
//write your code here
sortedt:`time xasc t
uniquet : update `u#ID from sortedt
uniquePart: update `g#sym from uniquet
meta t
meta uniquet
meta uniquePart


c    | t f a
-----| -----
time | p   s
sym  | s    
ID   | i    
price| f    
c    | t f a
-----| -----
time | p   s
sym  | s    
ID   | i   u
price| f    
c    | t f a
-----| -----
time | p   s
sym  | s   g
ID   | i   u
price| f    


In [20]:
\t:1000 select avg price from t where sym=`JPM,time.time>09:00,ID within 500 600
\t:1000 select avg price from uniquePart where sym=`JPM,time.time>09:00,ID within 500 600

7718
6742
